In [2]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [3]:
# !pip install -q seaborn
# !pip install -q tabulate

In [4]:
import sys 
import os
import glob
import cv2
from PIL import Image
import numpy as np

from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import gezi

In [5]:
# !ls ./eval.naic/label
# !ls ./out

In [6]:
def eval(pred_dir, label_dir=None, src='v2', classes=None, round=2):
  SRC = src
  ROUND = round
  label_dir_ = label_dir
  if SRC == 'v2':
    #17 but 5,6 missing
    CLASSES = ['water', 'track_road', 'build', 'track_airport', 'other_park', 'other_playground', 'arable_natural', 'arable_greenhouse',
                'grass_natural', 'grass_greenbelt', 'forest_natural', 'forest_planted', 'bare_natural', 'bare_planted', 'other_other']
  #   label_dir = '../input/quarter/train/label'
    label_dir = './eval.naic/label'
  elif SRC == 'v1':
    CLASSES = ['water', 'track', 'build', 'arable', 'grass', 'forest', 'bare', 'other']
    label_dir = './eval.naic.v1/label'
    if ROUND == 2:
      m2 = {
        0: 0,
        1: 1,
        2: 2,
        3: 1,
        4: 7,
        5: 7,
        6: 3,
        7: 3,
        8: 4,
        9: 4,
        10: 5,
        11: 5,
        12: 6,
        13: 6,
        14: 7,
      }
    else:
      m2 = dict(zip(range(8), range(8)))
  elif SRC == 'ccf': # ccf data
    # 植被（标记1）、道路（标记2）、建筑（标记3）、水体（标记4）以及其他(标记0)
    CLASSES = ['background', 'vegetation', 'road', 'build', 'water']
    label_dir = './eval.all/label'
    if ROUND == 2:
      m2 = {
        0: 4,
        1: 2,
        2: 3,
        3: 2,
        4: 0,
        5: 0,
        6: 1,
        7: 1,
        8: 1,
        9: 1,
        10: 1,
        11: 1,
        12: 1,
        13: 1,
        14: 0,
      }
    else:
      m2 = {
        0: 4,
        1: 2,
        2: 3,
        3: 1,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
      }
  elif SRC == 'baidu': # baidu
    CLASSES = ['build', 'arable', 'forest', 'water', 'track', 'grass', 'other']
    label_dir = './eval.baidu/label'
    if ROUND == 2:
      m2 = {
        0: 3,
        1: 4,
        2: 0,
        3: 4,
        4: 6,
        5: 6,
        6: 1,
        7: 1,
        8: 5,
        9: 5,
        10: 2,
        11: 2,
        12: 6,
        13: 6,
        14: 6,
      }
    else:
      m2 = {
        0: 3,
        1: 4,
        2: 0,
        3: 1,
        4: 5,
        5: 2,
        6: 6,
        7: 6,
      }
  else:
    raise ValueError(SRC)

  NUM_CLASSES = len(CLASSES)

  if label_dir_:
    label_dir = label_dir_

  m = {}
  for i in range(17):
    if i < 4:
      m[i + 1] = i
    else:
      m[i + 1] = i - 2

  key_metric = 'FWIoU'

  pred_files = glob.glob(f'{pred_dir}/*')

  evaluator = Evaluator(CLASSES)
  class_mious = 0.

  t = tqdm(enumerate(pred_files), total=len(pred_files), ascii=True, desc= 'eval')
  for i, pred_file in t:
    file_name = os.path.basename(pred_file)
    label_file = f'{label_dir}/{file_name}'
    try:
      if SRC != 'baidu':
        label = cv2.imread(label_file, cv2.IMREAD_UNCHANGED).astype(np.int32)
      else:
        label = np.asarray(Image.open(label_file))
      pred = cv2.imread(pred_file,cv2.IMREAD_UNCHANGED).astype(np.int32)    
    except Exception as e:
      print(label_file)
      print(e)
      break

    if SRC == 'v2':
      f = np.vectorize(lambda x: m[x])
      label = f(label).astype(np.uint8)
      pred = f(pred).astype(np.uint8)
    else:
      if SRC == 'v1':
        label = (label / 100 - 1).astype(np.uint8)
        # pred = (pred / 100 - 1).astype(np.uint8)
      elif SRC == 'baidu':
        f = np.vectorize(lambda x: 0 if x == 255 else x)
        label = f(label).astype(np.uint8)

      f = np.vectorize(lambda x: m[x])
      pred = f(pred).astype(np.uint8)
      f = np.vectorize(lambda x: m2[x])
      pred = f(pred).astype(np.uint8)

    evaluator.eval_each(label[np.newaxis,:, :], pred[np.newaxis,:, :], metric=key_metric)

    binary_label = np.bincount(label.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    binary_pred = np.bincount(pred.reshape(-1), minlength=NUM_CLASSES).astype(np.bool)
    intersections = binary_label * binary_pred
    intersection = np.sum(intersections, axis=-1)
    union = np.sum(binary_label, axis=-1) + np.sum(binary_pred, axis=-1) - intersection
    class_miou = np.mean(intersection / union)

    class_mious += class_miou

    t.set_postfix({key_metric: evaluator.eval_once(key_metric), 'IMAGE/CLASS/MIoU': class_mious / (i + 1)})

  res = evaluator.eval_once()
  res['IMAGE/CLASS/MIoU'] = class_mious / i
  gezi.set('evaluator', evaluator)
  return res

In [7]:
# pred_dir = './out'
# res = eval(pred_dir)
# gezi.pprint_dict(res)

In [8]:
cd /home/featurize/data

/home/featurize/data


In [13]:
pred_dir = '/home/featurize/data/out'
SRC = 'v2'
ROUND = 2
label_dir = '/home/featurize/dataset/labels/'
res = eval(pred_dir, src=SRC, round=ROUND, label_dir=label_dir)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+-------------+------------------+-------------+---------------------+------------------+------------------------+----------------------+-------------------------+---------------------+-----------------------+----------------------+----------------------+--------------------+--------------------+-------------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/water |   IoU/track_road |   IoU/build |   IoU/track_airport |   IoU/other_park |   IoU/other_playground |   IoU/arable_natural |   IoU/arable_greenhouse |   IoU/grass_natural |   IoU/grass_greenbelt |   IoU/forest_natural |   IoU/forest_planted |   IoU/bare_natural |   IoU/bare_planted |   IoU/other_other |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+-------------+------------------+-------------+---------------------+------------------+------------------------+----------------------+-------------------------+------

In [ ]:
pred_dir = './out.v1'
SRC = 'v2'
ROUND = 2
label_dir = './eval.naic/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)

In [8]:
pred_dir = './out'
SRC = 'v2'
ROUND = 2
label_dir = './eval.naic/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+-------------+------------------+-------------+---------------------+------------------+------------------------+----------------------+-------------------------+---------------------+-----------------------+----------------------+----------------------+--------------------+--------------------+-------------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/water |   IoU/track_road |   IoU/build |   IoU/track_airport |   IoU/other_park |   IoU/other_playground |   IoU/arable_natural |   IoU/arable_greenhouse |   IoU/grass_natural |   IoU/grass_greenbelt |   IoU/forest_natural |   IoU/forest_planted |   IoU/bare_natural |   IoU/bare_planted |   IoU/other_other |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+-------------+------------------+-------------+---------------------+------------------+------------------------+----------------------+-------------------------+------

In [ ]:
pred_dir = './out.dv1.v1'
SRC = 'v1'
ROUND = 2
label_dir = './eval.naic.v1/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)

In [19]:
pred_dir = './out.dv1'
SRC = 'v1'
ROUND = 2
label_dir = './eval.naic.v1/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/water |   IoU/track |   IoU/build |   IoU/arable |   IoU/grass |   IoU/forest |   IoU/bare |   IoU/other |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------|
| 0.289742 | 0.259168 |    0.446317 |    0.409526 |    0.350916 |    0.272992 |    0.533428 |     0.286362 |    0.100029 |     0.298093 |  0.0512532 |     0.18027 |           0.473644 |
+----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------+


In [5]:
pred_dir = './out.auglevel4'
SRC = 'v1'
ROUND = 2
label_dir = './eval.naic.v1/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/water |   IoU/track |   IoU/build |   IoU/arable |   IoU/grass |   IoU/forest |   IoU/bare |   IoU/other |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------|
| 0.266661 | 0.243399 |    0.407373 |    0.379943 |    0.322582 |       0.281 |    0.494658 |     0.240387 |   0.0910705 |     0.271676 |  0.0804064 |    0.165416 |           0.459629 |
+----------+----------+-------------+-------------+-------------+-------------+-------------+--------------+-------------+--------------+------------+-------------+--------------------+


In [ ]:
pred_dir = './out.dv1'
SRC = 'v1'
ROUND = 2
label_dir = './eval.naic.v1/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)

In [ ]:
evaluator = gezi.get('evaluator')

In [ ]:
gezi.plot.confusion_matrix(evaluator.confusion_matrix, evaluator.class_names, 'true')

In [ ]:
gezi.plot.confusion_matrix(evaluator.confusion_matrix, evaluator.class_names, 'pred')

In [ ]:
cm = evaluator.confusion_matrix
trues = np.sum(cm, 1)
preds = np.sum(cm, 0)
total = np.sum(trues)
classes_ratio_true = trues / total
classes_ratio_pred = preds / total

In [ ]:
classes_ratio_true

In [ ]:
classes_ratio_pred

In [ ]:
evaluator.class_names

In [14]:
pred_dir = './out.ccf.naic.bs4'
SRC = 'ccf'
ROUND = 2
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.363666 | 0.247244 |    0.466082 |    0.530955 |         0.365293 |         0.211418 |   0.214867 |    0.441926 |  0.00271718 |           0.660227 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [15]:
pred_dir = './out.ccf.naic.bs36'
SRC = 'ccf'
ROUND = 2
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.363666 | 0.247244 |    0.466082 |    0.530955 |         0.365293 |         0.211418 |   0.214867 |    0.441926 |  0.00271718 |           0.660227 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [7]:
pred_dir = './out.ccf.naic'
SRC = 'ccf'
ROUND = 2
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.382492 | 0.266253 |    0.490279 |    0.550555 |         0.377365 |           0.2212 |   0.264922 |    0.464832 |  0.00294605 |           0.669002 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [6]:
pred_dir = './out.ccf.naic'
SRC = 'ccf'
ROUND = 2
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.381721 | 0.265681 |    0.489385 |    0.550038 |         0.376469 |         0.220758 |   0.264011 |    0.464225 |  0.00293971 |           0.668919 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [10]:
pred_dir = './out.ccf.naic'
SRC = 'ccf'
ROUND = 2
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.382492 | 0.266253 |    0.490279 |    0.550555 |         0.377365 |           0.2212 |   0.264922 |    0.464832 |  0.00294605 |           0.669002 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [ ]:
pred_dir = './out.ccf.v1'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)

In [8]:
pred_dir = './out.ccf.v2'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.797815 | 0.824612 |    0.887161 |     0.90685 |         0.808568 |         0.870755 |   0.829255 |    0.753473 |    0.861007 |           0.986286 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [9]:
pred_dir = './out.ccf.v2'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.791098 | 0.819253 |    0.882961 |    0.902221 |         0.802491 |         0.862314 |   0.825502 |    0.745267 |    0.860693 |           0.982449 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [12]:
pred_dir = './out.ccf2'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.791098 | 0.819253 |    0.882961 |    0.902221 |         0.802491 |         0.862314 |   0.825502 |    0.745267 |    0.860693 |           0.982449 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [13]:
pred_dir = './out.ccf'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.797815 | 0.824612 |    0.887161 |     0.90685 |         0.808568 |         0.870755 |   0.829255 |    0.753473 |    0.861007 |           0.986286 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [14]:
pred_dir = './out.ccf'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)



 +----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+
|    FWIoU |     MIoU |   ACC/pixel |   ACC/class |   IoU/background |   IoU/vegetation |   IoU/road |   IoU/build |   IoU/water |   IMAGE/CLASS/MIoU |
|----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------|
| 0.797815 | 0.824612 |    0.887161 |     0.90685 |         0.808568 |         0.870755 |   0.829255 |    0.753473 |    0.861007 |           0.986286 |
+----------+----------+-------------+-------------+------------------+------------------+------------+-------------+-------------+--------------------+


In [ ]:
pred_dir = './out.ccf'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)

In [ ]:
pred_dir = './out.ccf2'
SRC = 'ccf'
ROUND = 1
label_dir = './eval.all/label'
res = eval(pred_dir, src=SRC, round=ROUND)
gezi.pprint_dict(res)